# js

> JavaScript generation for the Web Audio API manager

In [ ]:
#| default_exp js

In [ ]:
#| export
from cjm_fasthtml_web_audio.models import WebAudioConfig, WebAudioHtmlIds

## State Initialization

Generates the JS state object for a Web Audio manager instance.

In [ ]:
#| export
def generate_state_init(
    config: WebAudioConfig,  # Instance configuration
) -> str:  # JS code that initializes the state object
    """Generate JS code to initialize the namespaced state object."""
    sk = config.state_key
    
    speed_fields = ""
    if config.enable_speed:
        speed_fields += f"\n    {sk}.playbackSpeed = 1.0;"
    if config.enable_replay:
        speed_fields += f"\n    {sk}.currentSegment = null;"
    if config.enable_auto_nav:
        speed_fields += f"\n    {sk}.autoNavigate = false;"
    
    return f"""// Web Audio Manager: {config.namespace}
    window.{sk} = window.{sk} || {{}};
    {sk}.buffers = [];
    {sk}.ctx = null;
    {sk}.currentSource = null;
    {sk}.loading = false;
    {sk}.error = null;
    {sk}.pendingPlay = null;
    {sk}.playTimeout = null;
    {sk}.lastPlayedIndex = null;{speed_fields}
    window.DEBUG_{config.namespace.upper()}_AUDIO = false;"""

## Init Audio

Reads audio URLs from a hidden input, fetches all files, and decodes them in parallel via `Promise.all()`.

In [ ]:
#| export
def generate_init_audio(
    config: WebAudioConfig,  # Instance configuration
) -> str:  # JS init function
    """Generate JS function that loads and decodes audio files in parallel."""
    sk = config.state_key
    ns = config.ns
    urls_input_id = WebAudioHtmlIds.audio_urls_input(config.namespace)
    dbg = f"DEBUG_{config.namespace.upper()}_AUDIO"
    
    return f"""window.init{ns}Audio = async function() {{
        var s = window.{sk};
        if (s.buffers.length > 0 || s.loading) return;
        
        var urlsInput = document.getElementById('{urls_input_id}');
        if (!urlsInput || !urlsInput.value) {{
            console.warn('[{ns}Audio] No audio URLs input found');
            return;
        }}
        
        var urls;
        try {{ urls = JSON.parse(urlsInput.value); }}
        catch(e) {{ console.error('[{ns}Audio] Invalid URL JSON:', e); return; }}
        
        if (!Array.isArray(urls) || urls.length === 0) {{
            console.warn('[{ns}Audio] Empty audio URLs array');
            return;
        }}
        
        s.loading = true;
        try {{
            s.ctx = s.ctx || new (window.AudioContext || window.webkitAudioContext)();
            
            if (window.{dbg}) console.log('[{ns}Audio] Loading ' + urls.length + ' audio file(s)...');
            
            var decodePromises = urls.map(function(url, idx) {{
                return fetch(url)
                    .then(function(r) {{ return r.arrayBuffer(); }})
                    .then(function(buf) {{ return s.ctx.decodeAudioData(buf); }})
                    .then(function(decoded) {{
                        if (window.{dbg}) console.log('[{ns}Audio] Decoded buffer ' + idx + ': ' +
                            decoded.duration.toFixed(1) + 's, ' + decoded.sampleRate + 'Hz, ' +
                            decoded.numberOfChannels + 'ch');
                        return {{ index: idx, buffer: decoded }};
                    }});
            }});
            
            var results = await Promise.all(decodePromises);
            
            // Store buffers at their original indices
            s.buffers = new Array(urls.length);
            results.forEach(function(r) {{ s.buffers[r.index] = r.buffer; }});
            
            if (window.{dbg}) console.log('[{ns}Audio] All ' + s.buffers.length + ' buffers loaded');
            
            // Play pending request if queued
            if (s.pendingPlay) {{
                var p = s.pendingPlay;
                s.pendingPlay = null;
                window.play{ns}Segment(p.bufferIndex, p.start, p.end, p.indicator);
            }}
        }} catch(e) {{
            s.error = e;
            console.error('[{ns}Audio] Load error:', e);
        }} finally {{
            s.loading = false;
        }}
    }};"""

## Stop Audio

In [ ]:
#| export
def generate_stop_audio(
    config: WebAudioConfig,  # Instance configuration
) -> str:  # JS stop function
    """Generate JS function that stops current playback."""
    sk = config.state_key
    ns = config.ns
    
    return f"""window.stop{ns}Audio = function() {{
        var s = window.{sk};
        if (s.currentSource) {{
            try {{ s.currentSource.stop(); }} catch(e) {{}}
            s.currentSource.disconnect();
            s.currentSource = null;
        }}
        if (s.playTimeout) {{
            clearTimeout(s.playTimeout);
            s.playTimeout = null;
        }}
    }};"""

## Play Segment

Plays a time range from a specific audio buffer. Supports optional playback speed and auto-navigate.

In [ ]:
#| export
def generate_play_segment(
    config: WebAudioConfig,  # Instance configuration
    nav_down_btn_id: str = "",  # Nav down button ID (for auto-navigate)
) -> str:  # JS play function
    """Generate JS function that plays a segment from a specific buffer."""
    sk = config.state_key
    ns = config.ns
    dbg = f"DEBUG_{config.namespace.upper()}_AUDIO"
    
    # Speed-aware timeout calculation
    if config.enable_speed:
        speed_line = f"var speed = s.playbackSpeed || 1.0;"
        rate_line = "src.playbackRate.value = speed;"
        timeout_expr = "(duration / speed) * 1000"
    else:
        speed_line = ""
        rate_line = ""
        timeout_expr = "duration * 1000"
    
    # Store current segment for replay
    replay_store = ""
    if config.enable_replay:
        replay_store = f"s.currentSegment = {{bufferIndex: bufferIndex, start: start, end: end, indicator: indicator}};"
    
    # Auto-navigate on completion
    auto_nav = ""
    if config.enable_auto_nav and nav_down_btn_id:
        auto_nav = f"""
            if (s.autoNavigate) {{
                var navBtn = document.getElementById('{nav_down_btn_id}');
                if (navBtn) navBtn.click();
            }}"""
    
    return f"""window.play{ns}Segment = function(bufferIndex, start, end, indicator) {{
        var s = window.{sk};
        
        // Queue if not loaded yet
        if (s.buffers.length === 0) {{
            s.pendingPlay = {{bufferIndex: bufferIndex, start: start, end: end, indicator: indicator}};
            window.init{ns}Audio();
            return;
        }}
        
        // Resume suspended context (autoplay policy)
        if (s.ctx && s.ctx.state === 'suspended') s.ctx.resume();
        
        // Stop previous
        window.stop{ns}Audio();
        
        // Validate buffer index
        var bi = bufferIndex || 0;
        if (bi < 0 || bi >= s.buffers.length) {{
            console.warn('[{ns}Audio] Invalid buffer index: ' + bi);
            return;
        }}
        
        var buffer = s.buffers[bi];
        var clampStart = Math.max(0, Math.min(start, buffer.duration));
        var clampEnd = Math.max(clampStart, Math.min(end, buffer.duration));
        var duration = clampEnd - clampStart;
        if (duration <= 0) return;
        
        {speed_line}
        {replay_store}
        
        var src = s.ctx.createBufferSource();
        src.buffer = buffer;
        {rate_line}
        src.connect(s.ctx.destination);
        s.currentSource = src;
        src.start(0, clampStart, duration);
        
        if (window.{dbg}) console.log('[{ns}Audio] Playing buffer ' + bi + ': ' +
            clampStart.toFixed(2) + 's -> ' + clampEnd.toFixed(2) + 's');
        
        s.playTimeout = setTimeout(function() {{
            if (indicator) indicator.style.visibility = 'hidden';{auto_nav}
        }}, {timeout_expr});
    }};"""

## Optional Features

Speed control, replay, and auto-navigate functions generated only when enabled.

In [ ]:
#| export
def generate_optional_features(
    config: WebAudioConfig,  # Instance configuration
) -> str:  # JS for optional feature functions (empty if none enabled)
    """Generate JS for optional features based on config flags."""
    sk = config.state_key
    ns = config.ns
    parts = []
    
    if config.enable_speed:
        parts.append(f"""window.set{ns}Speed = function(speed) {{
        window.{sk}.playbackSpeed = speed;
    }};""")
    
    if config.enable_replay:
        parts.append(f"""window.replay{ns}Segment = function() {{
        var s = window.{sk};
        if (s.currentSegment) {{
            s.lastPlayedIndex = null;
            var c = s.currentSegment;
            if (c.indicator) c.indicator.style.visibility = 'visible';
            window.play{ns}Segment(c.bufferIndex, c.start, c.end, c.indicator);
        }}
    }};""")
    
    if config.enable_auto_nav:
        parts.append(f"""window.set{ns}AutoNavigate = function(flag) {{
        window.{sk}.autoNavigate = flag;
    }};""")
    
    return "\n\n    ".join(parts)

## Focus Change Callback

Card-stack-compatible callback triggered on focus changes. Reads data attributes from the card element to determine which buffer and time range to play.

In [ ]:
#| export
def generate_focus_change(
    config: WebAudioConfig,  # Instance configuration
    focus_input_id: str,  # Hidden input ID for focused index
) -> str:  # JS focus change callback
    """Generate JS focus change callback for card stack integration."""
    sk = config.state_key
    ns = config.ns
    dbg = f"DEBUG_{config.namespace.upper()}_AUDIO"
    idx_attr = config.data_index_attr
    start_attr = config.data_start_attr
    end_attr = config.data_end_attr
    indicator_sel = config.indicator_selector
    
    return f"""window.on{ns}FocusChange = function(item, index, zoneId) {{
        var s = window.{sk};
        
        // Update hidden input
        var fi = document.getElementById('{focus_input_id}');
        if (fi) fi.value = index;
        
        var currentIndex = parseInt(item.dataset.segmentIndex || item.dataset.chunkIndex || index, 10);
        
        // Skip replay of same card
        if (s.lastPlayedIndex === currentIndex) {{
            if (window.{dbg}) console.log('[{ns}Audio] Same card, skipping replay');
            return;
        }}
        s.lastPlayedIndex = currentIndex;
        
        // Hide all indicators
        document.querySelectorAll('{indicator_sel}').forEach(function(el) {{
            el.style.visibility = 'hidden';
        }});
        
        // Read data attributes
        var startTime = parseFloat(item.dataset.{start_attr});
        var endTime = parseFloat(item.dataset.{end_attr});
        var bufferIndex = parseInt(item.dataset.{idx_attr} || '0', 10);
        
        if (isNaN(startTime) || isNaN(endTime) || endTime <= startTime) return;
        
        // Show indicator on current card
        var indicator = item.querySelector('{indicator_sel}');
        if (indicator) indicator.style.visibility = 'visible';
        
        window.play{ns}Segment(bufferIndex, startTime, endTime, indicator);
    }};"""

## HTMX AfterSettle Handler

Re-attaches after each card swap to trigger focus change callback when navigation completes.

In [ ]:
#| export
def generate_htmx_settle_handler(
    config: WebAudioConfig,  # Instance configuration
    card_stack_id: str,  # Card stack container ID
) -> str:  # JS HTMX afterSettle handler
    """Generate HTMX afterSettle handler for card stack navigation."""
    ns = config.ns
    
    return f"""(function() {{
        var handlerKey = '_{config.namespace}FocusSettleHandler';
        if (window[handlerKey]) {{
            document.body.removeEventListener('htmx:afterSettle', window[handlerKey]);
        }}
        window[handlerKey] = function(evt) {{
            var target = evt.detail.target || evt.target;
            var csEl = document.getElementById('{card_stack_id}');
            if (!csEl) return;
            if (target.id !== '{card_stack_id}' && !csEl.contains(target)) return;
            
            // Check if this zone is active
            if (window.kbNav) {{
                var kbState = window.kbNav.getState();
                if (kbState && kbState.activeZoneId !== '{card_stack_id}') {{
                    return;
                }}
            }}
            
            var focused = csEl.querySelector('[data-card-role="focused"]');
            if (focused) {{
                window.on{ns}FocusChange(focused, 0, '');
            }}
        }};
        document.body.addEventListener('htmx:afterSettle', window[handlerKey]);
    }})();"""

## Full Script Assembly

Combines all JS parts into a single string.

In [ ]:
#| export
def generate_web_audio_js(
    config: WebAudioConfig,  # Instance configuration
    focus_input_id: str,  # Hidden input ID for focused index
    card_stack_id: str,  # Card stack container ID
    nav_down_btn_id: str = "",  # Nav down button ID (for auto-navigate)
) -> str:  # Complete JS code for this instance
    """Generate the complete Web Audio API JS for a configured instance."""
    parts = [
        generate_state_init(config),
        generate_init_audio(config),
        generate_stop_audio(config),
        generate_play_segment(config, nav_down_btn_id),
        generate_focus_change(config, focus_input_id),
        generate_htmx_settle_handler(config, card_stack_id),
    ]
    
    optional = generate_optional_features(config)
    if optional:
        parts.append(optional)
    
    return "\n\n    ".join(parts)

## Tests

In [ ]:
# Test basic config generates expected function names
cfg = WebAudioConfig(namespace="test", indicator_selector=".test-indicator")
js = generate_web_audio_js(cfg, focus_input_id="test-focus", card_stack_id="test-cs")

assert "window._webAudio_test" in js
assert "window.initTestAudio" in js
assert "window.stopTestAudio" in js
assert "window.playTestSegment" in js
assert "window.onTestFocusChange" in js
assert "sd-test-audio-urls" in js
assert "Promise.all" in js

# No optional features by default
assert "setTestSpeed" not in js
assert "replayTestSegment" not in js
assert "setTestAutoNavigate" not in js
print("Basic config test passed")

Basic config test passed


In [ ]:
# Test full-featured config
cfg2 = WebAudioConfig(
    namespace="review",
    indicator_selector=".review-playing-indicator",
    enable_speed=True, enable_replay=True, enable_auto_nav=True,
)
js2 = generate_web_audio_js(
    cfg2, focus_input_id="review-focus", card_stack_id="review-cs",
    nav_down_btn_id="review-nav-down",
)

assert "window._webAudio_review" in js2
assert "window.setReviewSpeed" in js2
assert "window.replayReviewSegment" in js2
assert "window.setReviewAutoNavigate" in js2
assert "playbackSpeed" in js2
assert "review-nav-down" in js2
assert "autoNavigate" in js2
print("Full-featured config test passed")

Full-featured config test passed


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()